In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [2]:
rls = pd.read_csv('data/RLS_Exterior_Features_2024_08_27.csv')
rls.head()

,listing_key,listing_id,property_type,property_sub_type,originating_system_i_d,public_remarks,building_exterior_features,exterior_features
0,RLMX-95264,NaN,Residential Lease,Apartment,RLMX,Welcome to your newly renovated apartment in t...,NaN,NaN
1,OLRS-00011505707,NaN,Residential Lease,Apartment,OLRS,NEWLY RENOVATED TWO BEDROOM!<br>LOCATED AT 211...,[],[]
2,RLMX-95263,NaN,Residential Lease,Apartment,RLMX,Welcome to your newly renovated apartment in t...,NaN,NaN
3,RPLU-5122948400,NaN,Residential Lease,Apartment,RPLU,<p>Make this spacious 2 bedroom/1 bath apartme...,[],['None']
4,RPLU-5122893685,NaN,Residential,Unit Duplex,RPLU,Experience the epitome of luxury living in thi...,['Building Roof Deck'],['None']


In [3]:
pw = pd.read_csv('data/dbeaver_exports/nyc_outdoor_space.csv')
for i in range(2,42):
    pw = pd.concat([pw, pd.read_csv(f'data/dbeaver_exports/nyc_outdoor_space_{i}.csv')])
pw.head()

,id,rebny_id,source,address,property_type_code,description,balcony,exterior_features,building_exterior_features,outdoor_space,balcony.1,terrace,garden
0,2876724,CORC-5974265,rls,220 West 14th Street,400,PLEASE READ - Pricing listed is net effective ...,NaN,NaN,NaN,NaN,NaN,True,True
1,640685,1072736,real_plus,230 East 15th Street,550,NaN,NaN,NaN,{},NaN,NaN,True,NaN
2,640685,1072736,real_plus,230 East 15th Street,550,NaN,NaN,{Lighting},NaN,NaN,NaN,True,NaN
3,2298749,3047947,mlsli,200 S Middle Neck Road,550,Feels Like A House....Totally Gut Renovated 2 ...,NaN,NaN,NaN,True,NaN,NaN,True
4,2922021,2912100,mlsli,61-55 98th Street,550,"Beautiful Co-Op Apt - 2 Nice Size Br, With Lar...",NaN,NaN,NaN,True,NaN,NaN,NaN


In [4]:
def private_or_public(s: str) -> bool:
    s = s.lower()
    retlist = []
    if 'balcony' in s:
        retlist.append('Balcony')
    if 'balconies' in s:
        retlist.append('Balcony')
    if 'private terrace' in s:
        retlist.append('Balcony')
    if 'private roof' in s:
        retlist.append('Roof Deck')
    #elif 'private outdoor' in s:
     #   return True
    if 'private patio' in s:
        retlist.append('Private Yard')
    if 'wrap terrace' in s:
        retlist.append('Balcony')
    if 'private' in s and ('backyard' in s or 'front yard' in s or 'frontyard' in s):
        retlist.append('Private Yard')
    if len(retlist)>0:
        return retlist
    else:
        return False
    
def private_or_public_2(s: str) -> bool:
    s = s.lower()
    retlist = []
    if 'garden' in s:
        retlist.append('Building Garden')
    if 'courtyard' in s:
        retlist.append('Building Courtyard')
    if 'roof' in s:
        retlist.append('Building Roof Deck')
    if len(retlist)>0:
        return retlist
    else:
        return False

In [5]:
rls['private_outdoor_space'] = [False]*len(rls)
rls['private_listing_desc'] = [False]*len(rls)
rls['private_exterior_features'] = [False]*len(rls)
rls['public_outdoor_space'] = [False]*len(rls)
rls['public_listing_desc'] = [False]*len(rls)
rls['public_exterior_features'] = [False]*len(rls)
pw['public_or_private'] = ['public']*len(pw)
pw['decision_source'] = ['n/a']*len(pw)

In [6]:
import ast
count = 0
ret_exterior_features = []
ret_building_features = []
for index, row in tqdm(rls.iterrows(), total=len(rls)):
    #if not (pd.isna(row['balcony']) or row['balcony'] == False):
     #   rls.loc[index, 'public_or_private'] = 'private'
      #  rls.loc[index, 'decision_source'] = 'balcony_field'
    new_exterior_features = []
    new_building_exterior_features = []
    try:
        exterior_features = ast.literal_eval(row['exterior_features'])
        #print(exterior_features)
    except:
        exterior_features = [None]
    try:
        building_exterior_features = ast.literal_eval(row['building_exterior_features'])
        #print(exterior_features)
    except:
        building_exterior_features = [None]
    #if row['property_sub_type'] in ['Single-Family Townhouse', 'Multi-Family Townhouse',
     #  'Single Family Residence', 'Townhouse']:
      #  rls.loc[index, 'public_or_private'] = 'private'
       # rls.loc[index, 'decision_source'] = 'property_type_field'
    if ('Balcony' in exterior_features or 
          #'Courtyard' in exterior_features or 
          #'Garden' in exterior_features or 
          'Juliet Balcony' in exterior_features or 
          #'Private Entrance' in exterior_features or 
          'Private Yard' in exterior_features): # or 
          #'Uncovered Courtyard' in exterior_features):
        new_exterior_features = exterior_features
        rls.loc[index, 'private_outdoor_space'] = True
        rls.loc[index, 'private_exterior_features'] = True
    in_desc = private_or_public(str(row['public_remarks']))
    if in_desc:
        rls.loc[index, 'private_outdoor_space'] = True
        rls.loc[index, 'private_listing_desc'] = True
        for d in in_desc:
            if d not in new_exterior_features:
                new_exterior_features.append(d)
    #if row['public_or_private'] == 'private':
    #    continue
    #else:
    building_exterior_features += exterior_features
    if ('Building Garden' in building_exterior_features or 
        'Building Roof Deck' in building_exterior_features or
        'Building Courtyard' in building_exterior_features or
        'Outdoor Grill' in building_exterior_features or
        'Outdoor Kitchen' in building_exterior_features):
        new_building_exterior_features = [x for x in building_exterior_features if (x and x != 'None' and x not in new_exterior_features)]
        rls.loc[index, 'public_outdoor_space'] = True
        rls.loc[index, 'public_listing_desc'] = True
    
    in_desc = private_or_public_2(str(row['public_remarks']))
    if in_desc:
        rls.loc[index, 'public_outdoor_space'] = True
        rls.loc[index, 'public_exterior_features'] = True
        for d in in_desc:
            if d not in new_building_exterior_features and d != 'None' and d not in new_exterior_features:
                new_building_exterior_features.append(d)
    ret_exterior_features.append(new_exterior_features)
    ret_building_features.append(new_building_exterior_features)
    #if len(new_exterior_features)==0:
     #   count += 1
      #  rls.loc[index, 'public_or_private'] = 'public'
       # rls.loc[index, 'decision_source'] = np.nan

rls['new_exterior_features'] = ret_exterior_features
rls['new_building_exterior_features'] = ret_building_features
rls.head()

  0%|          | 0/79748 [00:00<?, ?it/s]

,listing_key,listing_id,property_type,property_sub_type,originating_system_i_d,public_remarks,building_exterior_features,exterior_features,private_outdoor_space,private_listing_desc,private_exterior_features,public_outdoor_space,public_listing_desc,public_exterior_features,new_exterior_features,new_building_exterior_features
0,RLMX-95264,NaN,Residential Lease,Apartment,RLMX,Welcome to your newly renovated apartment in t...,NaN,NaN,False,False,False,False,False,False,[],[]
1,OLRS-00011505707,NaN,Residential Lease,Apartment,OLRS,NEWLY RENOVATED TWO BEDROOM!<br>LOCATED AT 211...,[],[],False,False,False,False,False,False,[],[]
2,RLMX-95263,NaN,Residential Lease,Apartment,RLMX,Welcome to your newly renovated apartment in t...,NaN,NaN,False,False,False,False,False,False,[],[]
3,RPLU-5122948400,NaN,Residential Lease,Apartment,RPLU,<p>Make this spacious 2 bedroom/1 bath apartme...,[],['None'],False,False,False,False,False,False,[],[]
4,RPLU-5122893685,NaN,Residential,Unit Duplex,RPLU,Experience the epitome of luxury living in thi...,['Building Roof Deck'],['None'],True,True,False,True,True,True,"[Balcony, Private Yard]",[Building Roof Deck]


In [7]:
# rls['exterior_features_changed'] = rls['exterior_features'] == rls['new_exterior_features']
rls['building_exterior_features_changed'] = rls['building_exterior_features'] == rls['new_building_exterior_features']
rls.sample(frac=0.5, random_state=42).to_csv('data/rls_output.csv', index=False)
#[(rls['private_outdoor_space']) | (rls['private_listing_desc']) | (rls['private_exterior_features']) |
#    (rls['public_outdoor_space']) | (rls['public_listing_desc']) | (rls['public_exterior_features']) ]

#pw.sample(frac=0.1, random_state=42).to_csv('data/pw_output.csv', index=False)

In [7]:
count = 0
ret_exterior_features = []
ret_building_features = []
for index, row in tqdm(pw.iterrows(), total=len(pw)):
    new_exterior_features = []
    new_building_exterior_features = []
    if not (pd.isna(row['balcony']) or row['balcony'] == False):
        pw.loc[index, 'public_or_private'] = 'private'
        pw.loc[index, 'decision_source'] = 'balcony_field'
        new_exterior_features.append('Balcony')
    #elif row['property_type_code'] in [560, 580, 590, 600, 610]:
     #   pw.loc[index, 'public_or_private'] = 'private'
      #  pw.loc[index, 'decision_source'] = 'property_type_field'
    in_desc = private_or_public(str(row['description']))
    if in_desc:
        pw.loc[index, 'public_or_private'] = 'private'
        pw.loc[index, 'decision_source'] = 'listing_description'
        for d in in_desc:
            if d not in new_exterior_features:
                new_exterior_features.append(d)
    if len(new_exterior_features) == 0:

        if not (pd.isna(row['terrace']) or row['terrace'] == False):
            pw.loc[index, 'public_or_private'] = 'public'
            pw.loc[index, 'decision_source'] = 'terrace_field'
            new_building_exterior_features.append('Building Roof Deck')
        if not (pd.isna(row['garden']) or row['garden'] == False):
            pw.loc[index, 'public_or_private'] = 'public'
            pw.loc[index, 'decision_source'] = 'garden_field'
            new_building_exterior_features.append('Building Garden')
        if not (pd.isna(row['outdoor_space']) or row['outdoor_space'] == False):
            pw.loc[index, 'public_or_private'] = 'public'
            pw.loc[index, 'decision_source'] = 'outdoor_space_field'
            new_building_exterior_features.append('Other')
        in_desc = private_or_public_2(str(row['description']))
        if in_desc:
            rls.loc[index, 'public_or_private'] = 'public'
            rls.loc[index, 'decision_source'] = 'listing_description'
            for d in in_desc:
                if d not in new_building_exterior_features:
                    new_building_exterior_features.append(d)
    #if row['public_or_private'] == 'private':
     #   continue
    #else:
    ret_building_features.append(new_building_exterior_features)
    ret_exterior_features.append(new_exterior_features)
    
pw['new_building_exterior_features'] = ret_building_features
pw['new_exterior_features'] = ret_exterior_features
pw.head()

  0%|          | 0/380628 [00:00<?, ?it/s]

,id,rebny_id,source,address,property_type_code,description,balcony,exterior_features,building_exterior_features,outdoor_space,balcony.1,terrace,garden,public_or_private,decision_source,new_building_exterior_features,new_exterior_features
0,2876724,CORC-5974265,rls,220 West 14th Street,400,PLEASE READ - Pricing listed is net effective ...,NaN,NaN,NaN,NaN,NaN,True,True,private,listing_description,"[Building Roof Deck, Building Garden]",[]
1,640685,1072736,real_plus,230 East 15th Street,550,NaN,NaN,NaN,{},NaN,NaN,True,NaN,private,listing_description,[Building Roof Deck],[]
2,640685,1072736,real_plus,230 East 15th Street,550,NaN,NaN,{Lighting},NaN,NaN,NaN,True,NaN,private,listing_description,[Building Roof Deck],[]
3,2298749,3047947,mlsli,200 S Middle Neck Road,550,Feels Like A House....Totally Gut Renovated 2 ...,NaN,NaN,NaN,True,NaN,NaN,True,private,listing_description,"[Building Garden, Other]",[]
4,2922021,2912100,mlsli,61-55 98th Street,550,"Beautiful Co-Op Apt - 2 Nice Size Br, With Lar...",NaN,NaN,NaN,True,NaN,NaN,NaN,private,listing_description,[Other],[]


In [9]:
import ast
retlist = []
for i in rls['exterior_features']:
    if not pd.isna(i):
        retlist += ast.literal_eval(i)
set(retlist)

{'Awning(s)',
 'Balcony',
 'Barbecue',
 'Basketball Court',
 'Building Courtyard',
 'Building Garden',
 'Building Roof Deck',
 'Built-in Barbecue',
 'Courtyard',
 'Fire Pit',
 'Garden',
 'Gas Grill',
 'Juliet Balcony',
 'Lighting',
 'None',
 'Other',
 'Outdoor Grill',
 'Outdoor Kitchen',
 'Outdoor Shower',
 'Playground',
 'Private Entrance',
 'Private Yard',
 'Roof Deck',
 'Storage',
 'Tennis Court(s)',
 'Uncovered Courtyard'}

In [10]:
import ast
retlist = []
for i in rls['building_exterior_features']:
    if not pd.isna(i):
        retlist += ast.literal_eval(i)
set(retlist)

{'Awning(s)',
 'Balcony',
 'Barbecue',
 'Basketball Court',
 'Building Garden',
 'Building Roof Deck',
 'Built-in Barbecue',
 'Courtyard',
 'Fire Pit',
 'Garden',
 'Gas Grill',
 'Lighting',
 'None',
 'Other',
 'Outdoor Grill',
 'Outdoor Kitchen',
 'Outdoor Shower',
 'Playground',
 'Storage',
 'Tennis Court(s)',
 'Uncovered Courtyard'}

In [11]:
rls['property_type'].unique()

array(['Residential', 'Residential Lease'], dtype=object)

In [12]:
rls['property_sub_type'].unique()

array(['Unit Duplex', 'Apartment', 'Unit Triplex',
       'Single-Family Townhouse', 'Multi-Family Townhouse',
       'Multi Family', 'Loft', 'Single Family Residence', 'Triplex',
       'Office', 'Mixed Use', 'Duplex', 'Townhouse'], dtype=object)

In [13]:
rls.merge(pw, right_on='rebny_id', left_on='listing_key', suffixes=('_rls', '_pw'))[['rebny_id', 'property_type',
                                                                                     'property_sub_type', 'originating_system_i_d', 
                                                                                     'description', 'balcony', 'building_exterior_features_rls', 
                                                                                     'exterior_features_rls', 'public_or_private_rls', 
                                                                                     'decision_source_rls', 'building_exterior_features_pw', 
                                                                                     'exterior_features_pw', 'public_or_private_pw',
                                                                                     'decision_source_pw', 'new_exterior_features_pw', 'new_building_exterior_features_pw'
                                                                                     , 'new_exterior_features_rls', 'new_building_exterior_features_rls']].to_csv('data/rls_pw_output.csv', index=False)